In [308]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torch.nn.init
import torchvision

In [309]:
device = 'cuda' if torch.cuda.is_available else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed(777)

In [310]:
#model parameter
training_epochs = 15

In [311]:
train_data = dsets.ImageFolder(root = 'C:/Users/ehrme/모두의 딥러닝 torch/train_data/',
                              transform = transforms.ToTensor())

trans = transforms.Compose([
    transforms.Resize((128, 64))
])

test = torchvision.datasets.ImageFolder(root = 'C:/Users/ehrme/모두의 딥러닝 torch/test_origin/',
                             transform = trans)

for num, value in enumerate(test):
    data, label = value
    print(num, data, label)
    if (label == 1):
        data.save('C:/Users/ehrme/모두의 딥러닝 torch/test_data/KerasDeeplearning/%d_%d.jpeg'%(num, label))
    else:
        data.save('C:/Users/ehrme/모두의 딥러닝 torch/test_data/KoreanEmbedding/%d_%d.jpeg'%(num, label))

0 <PIL.Image.Image image mode=RGB size=64x128 at 0x1954F0471C8> 0
1 <PIL.Image.Image image mode=RGB size=64x128 at 0x1954FD991C8> 0
2 <PIL.Image.Image image mode=RGB size=64x128 at 0x1954FDD4348> 1
3 <PIL.Image.Image image mode=RGB size=64x128 at 0x1954FDD4E88> 1


In [320]:
test_data = dsets.ImageFolder(root = 'C:/Users/ehrme/모두의 딥러닝 torch/test_data/',
                              transform = transforms.ToTensor())

test_set = torch.utils.data.DataLoader(dataset = test_data, batch_size = len(test_data))

In [321]:
data_loader = torch.utils.data.DataLoader(dataset = train_data,
                                         batch_size = 8,
                                          shuffle = True,
                                          drop_last = True)

In [322]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, 5, stride = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, 5, stride = 1),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.fc = nn.Linear(6032, 120)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(120, 2)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
            
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc2(out)

        return out

In [323]:
model = CNN()
print(model)

CNN(
  (layer1): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=6032, out_features=120, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=120, out_features=2, bias=True)
)


In [324]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [325]:
#training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('Epoch: ', epoch+1, 'Cost: ', avg_cost)
print('Learning Finished!')

Epoch:  1 Cost:  tensor(0.7477, grad_fn=<AddBackward0>)
Epoch:  2 Cost:  tensor(0.6709, grad_fn=<AddBackward0>)
Epoch:  3 Cost:  tensor(0.6178, grad_fn=<AddBackward0>)
Epoch:  4 Cost:  tensor(0.5390, grad_fn=<AddBackward0>)
Epoch:  5 Cost:  tensor(0.5174, grad_fn=<AddBackward0>)
Epoch:  6 Cost:  tensor(0.4990, grad_fn=<AddBackward0>)
Epoch:  7 Cost:  tensor(0.4043, grad_fn=<AddBackward0>)
Epoch:  8 Cost:  tensor(0.3830, grad_fn=<AddBackward0>)
Epoch:  9 Cost:  tensor(0.3223, grad_fn=<AddBackward0>)
Epoch:  10 Cost:  tensor(0.1901, grad_fn=<AddBackward0>)
Epoch:  11 Cost:  tensor(0.1083, grad_fn=<AddBackward0>)
Epoch:  12 Cost:  tensor(0.0758, grad_fn=<AddBackward0>)
Epoch:  13 Cost:  tensor(0.0837, grad_fn=<AddBackward0>)
Epoch:  14 Cost:  tensor(0.0555, grad_fn=<AddBackward0>)
Epoch:  15 Cost:  tensor(0.0417, grad_fn=<AddBackward0>)
Learning Finished!


In [326]:
with torch.no_grad():
  #test는 학습할 필요가 없으므로 no gradient
  for num, data in enumerate(test_set):
    imgs, label = data
    prediction = model(imgs)
    correct_prediction = torch.argmax(prediction, 1) == label
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

Accuracy:  1.0
